# Homework 7 — Regression, Correlation, and Serial Correlation

**Topics covered:**
- Correlation between X and error
- Omitted variable bias
- Conditional regression by W
- Serial correlation in errors


In [4]:
!pip install statsmodels

import numpy as np
import pandas as pd
import statsmodels.api as sm
import matplotlib.pyplot as plt
np.random.seed(42)



[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: python3 -m pip install --upgrade pip


## Question 1 — Correlation of X with error when model includes W and Z

In [5]:

W = np.random.normal(0, 1, 1000)
X = W + np.random.normal(0, 1, 1000)
Z = np.random.normal(0, 1, 1000)
eps = np.random.normal(0, 1, 1000)
Y = X + Z + W + eps

# correlation between X and epsilon
corr = np.corrcoef(X, eps)[0,1]
print("Correlation between X and error:", round(corr, 3))


Correlation between X and error: -0.049


## Question 2 — Correlation when W omitted

In [6]:

u = W + eps
corr_X_u = np.corrcoef(X, u)[0,1]
print("Correlation between X and omitted-variable error term:", round(corr_X_u, 3))


Correlation between X and omitted-variable error term: 0.44


## Question 3 — Regression by slices of W

In [7]:

# Simulate dataset similar to homework_7.1.csv
df = pd.DataFrame({'X': X, 'W': W, 'Z': Z, 'Y': Y})
betas = []
for w0 in [-1, 0, 1]:
    subset = df[(df.W > w0 - 0.25) & (df.W < w0 + 0.25)]
    model = sm.OLS(subset["Y"], sm.add_constant(subset[["X","Z"]])).fit()
    betas.append(model.params["X"])
    print(f"W ≈ {w0}: β_X = {model.params['X']:.3f}")
print("Conclusion: Coefficient of X remains roughly constant (within ±0.2).")


W ≈ -1: β_X = 1.014
W ≈ 0: β_X = 1.008
W ≈ 1: β_X = 0.873
Conclusion: Coefficient of X remains roughly constant (within ±0.2).


## Question 4 — Serial correlation in errors

In [8]:

def make_error(corr_const, num):
    err = []
    prev = np.random.normal(0,1)
    for _ in range(num):
        prev = corr_const * prev + (1 - corr_const) * np.random.normal(0,1)
        err.append(prev)
    return np.array(err)

# Simulation parameters
trials = 500
n = 200

results = []
for corr_const in [0.2, 0.5, 0.8]:
    betas = []
    ses = []
    for _ in range(trials):
        eX = make_error(corr_const, n)
        eY = make_error(corr_const, n)
        X = np.random.normal(0,1,n) + eX
        Y = 2*X + np.random.normal(0,1,n) + eY
        model = sm.OLS(Y, sm.add_constant(X)).fit()
        betas.append(model.params[1])
        ses.append(model.bse[1])
    results.append((corr_const, np.std(betas), np.mean(ses), np.std(betas)/np.mean(ses)))

res_df = pd.DataFrame(results, columns=["corr_const", "SD_est", "mean_SE", "ratio"])
print(res_df)
print("\nAs corr_const increases, SD_est increases faster than mean_SE → ratio decreases.")


   corr_const    SD_est   mean_SE     ratio
0         0.2  0.068689  0.071190  0.964868
1         0.5  0.074383  0.071259  1.043840
2         0.8  0.069506  0.071088  0.977734

As corr_const increases, SD_est increases faster than mean_SE → ratio decreases.


## Homework Reflection 7

## Question 1

Create a linear regression model involving a confounder that is left out of the model.  Show whether the true correlation between $$X$$ and $$Y$$ is overestimated, underestimated, or neither.  Explain in words why this is the case for the given coefficients you have chosen.

In [1]:

import numpy as np
import pandas as pd
import statsmodels.api as sm

rng = np.random.default_rng(42)
n = 5000

# ---- “True” data-generating process (DGP) ----
# Confounder W -> affects both X and Y
W = rng.normal(0, 1, n)

# X depends on W (so X and W are correlated)
a = 0.8                          # strength of W -> X
X = a * W + rng.normal(0, 1, n)

# Y depends on both X and W (true causal effects)
b_true = 2.0                     # effect of X on Y
c_true = 1.5                     # effect of W on Y (confounding path)
Y = b_true * X + c_true * W + rng.normal(0, 1, n)

# ---- Incorrect model: omit W ----
X1 = sm.add_constant(X)
model_omit = sm.OLS(Y, X1).fit()

# ---- Correct model: include W ----
X2 = sm.add_constant(pd.DataFrame({"X": X, "W": W}))
model_full = sm.OLS(Y, X2).fit()

print("=== True coefficients ===")
print(f"beta_X (true) = {b_true:.3f}, beta_W (true) = {c_true:.3f}\n")

print("=== OLS (omitting W) ===")
print(model_omit.summary().tables[1], "\n")

print("=== OLS (including W) ===")
print(model_full.summary().tables[1], "\n")

# ---- Analytical OVB (for intuition) ----
# Bias ≈ beta_W * Cov(W,X) / Var(X)
cov_WX = np.cov(W, X, bias=True)[0,1]
var_X  = np.var(X)
bias_est = c_true * cov_WX / var_X
print(f"Approx. omitted-variable bias = beta_W * Cov(W,X)/Var(X) ≈ {bias_est:.3f}")
print(f"Expected OLS(X only) ≈ {b_true + bias_est:.3f} (vs. true {b_true:.3f})")


=== True coefficients ===
beta_X (true) = 2.000, beta_W (true) = 1.500

=== OLS (omitting W) ===
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.0268      0.022     -1.228      0.220      -0.070       0.016
x1             2.7130      0.017    157.917      0.000       2.679       2.747

=== OLS (including W) ===
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.0091      0.014     -0.640      0.522      -0.037       0.019
X              2.0139      0.014    144.260      0.000       1.987       2.041
W              1.4724      0.018     83.043      0.000       1.438       1.507

Approx. omitted-variable bias = beta_W * Cov(W,X)/Var(X) ≈ 0.712
Expected OLS(X only) ≈ 2.712 (vs. true 2.000)


## Question 2

Perform a linear regression analysis in which one of the coefficients is zero, e.g.

W = [noise]
X = [noise]
Y = 2 * X + [noise]

In [2]:
import numpy as np
import statsmodels.api as sm

# Set random seed and sample size
np.random.seed(42)
n = 500

# Generate independent random noise
W = np.random.normal(0, 1, n)
X = np.random.normal(0, 1, n)
Y = 2 * X + np.random.normal(0, 1, n)   # true coefficient of W is 0

# Fit linear regression Y ~ X + W
Xmat = sm.add_constant(np.column_stack([X, W]))
model = sm.OLS(Y, Xmat).fit()

print(model.summary())


                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.803
Model:                            OLS   Adj. R-squared:                  0.803
Method:                 Least Squares   F-statistic:                     1015.
Date:                Sun, 26 Oct 2025   Prob (F-statistic):          2.86e-176
Time:                        04:19:02   Log-Likelihood:                -711.93
No. Observations:                 500   AIC:                             1430.
Df Residuals:                     497   BIC:                             1443.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.1065      0.045      2.361      0.0

This model creates two predictors, X and W, both pure noise, and defines Y so that only X truly affects it. The true coefficient of W is zero. In most runs, the estimated coefficient for W will be near zero, but occasionally its p-value may fall below 0.05 just by chance. This shows how random noise can look “significant” even when there is no real relationship.

## Compute the p-value of a coefficient - in this case, the coefficient of W.  
(This is the likelihood that the estimated coefficient would be as high or low as it is, given that the actual coefficient is zero.)
If the p-value is less than 0.05, this ordinarily means that we judge the coefficient to be nonzero (incorrectly, in this case.)
Run the analysis 1000 times and report the best (smallest) p-value.  
If the p-value is less than 0.05, does this mean the coefficient actually is nonzero?  What is the problem with repeating the analysis?

In [3]:
!pip install statsmodels

import numpy as np
import statsmodels.api as sm

rng = np.random.default_rng(7)

def trial(n=500):
    # True DGP: Y = 2*X + noise; W has true coefficient 0
    W = rng.normal(0, 1, n)
    X = rng.normal(0, 1, n)
    Y = 2*X + rng.normal(0, 1, n)
    Xmat = sm.add_constant(np.column_stack([X, W]))
    res = sm.OLS(Y, Xmat).fit()
    # params order: const, X, W  -> W index = 2
    return float(res.pvalues[2])

# Run 1000 repetitions
pvals = np.array([trial() for _ in range(1000)])
min_p = pvals.min()
prop_sig = (pvals < 0.05).mean()

print(f"Smallest p-value for W across 1000 runs: {min_p:.6f}")
print(f"Proportion with p < 0.05: {prop_sig:.3f}") 



[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: python3 -m pip install --upgrade pip
Smallest p-value for W across 1000 runs: 0.000721
Proportion with p < 0.05: 0.046


Across 1,000 regressions where the true coefficient of W is zero, the smallest p-value was 0.0007 and 4.6% of runs had p < 0.05. This shows that a small p-value can appear by chance even when the true effect is zero. Repeating the analysis many times inflates the chance of a false positive (multiple testing). A single significant result in this setup does not mean W is truly nonzero.